# Grouping and CASE

Course videos: https://www.red-gate.com/hub/university/courses/t-sql/tsql-for-beginners

Course scripts: https://litknd.github.io/TSQLBeginners 

# Setup: Create a database

We create a super-simple database to illustrate grouping.

In [ ]:
USE master;
GO
IF DB_ID('TSQLSchool') IS NOT NULL
    ALTER DATABASE TSQLSchool SET SINGLE_USER WITH ROLLBACK IMMEDIATE
DROP DATABASE TSQLSchool;
GO
CREATE DATABASE TSQLSchool;
GO
USE TSQLSchool;
GO

/* Let's invent a new worst practice: 
name your tables after twitter accounts, including leading the
table name with @, which usually indicates a variable  */

CREATE TABLE dbo.[@dog_rates]
(
    doggo VARCHAR(128) NOT NULL,
    ratingoutof10 INT NOT NULL,
    ratedby VARCHAR(128) NOT NULL
);
GO

INSERT dbo.[@dog_rates]
(
    doggo,
    ratingoutof10,
    ratedby
)
VALUES
('Stormy', 14, 'Kendar'),
('Stormy', 13, 'Fletcher'),
('Fletcher', 14, 'Kendar'),
('Mister', 14, 'Mister'),
('Mister', 14, 'Kendar');
GO

SELECT *
FROM dbo.[@dog_rates];
GO

# GROUP BY

## Logical processing order 

* FROM
* ON
* JOIN
* WHERE
* <mark>GROUP BY</mark>
* WITH CUBE or WITH ROLLUP
* HAVING
* SELECT
* DISTINCT
* ORDER BY
* TOP

https://docs.microsoft.com/en-us/sql/t-sql/queries/select-transact-sql 

## GROUP BY and COUNT()

![group by and count](images/group_count.png)

In [ ]:
USE TSQLSchool;
GO
SELECT
    doggo,
    COUNT(*) as tot
FROM dbo.[@dog_rates]
GROUP BY doggo;
GO

## GROUP BY and MAX()

![group by and max](images/group_max.png)

In [ ]:
USE TSQLSchool;
GO
SELECT
    doggo,
    MAX(ratingoutof10) as tot
FROM dbo.[@dog_rates]
GROUP BY doggo;
GO

## GROUP BY and MIN()

![group by and min](images/group_min.png)

In [ ]:
USE TSQLSchool;
GO
SELECT
    doggo,
    MIN(ratingoutof10) as tot
FROM dbo.[@dog_rates]
GROUP BY doggo;
GO

## GROUP BY and SUM()

![group by and sum](images/group_sum.png)

In [ ]:
USE TSQLSchool;
GO
SELECT
    doggo,
    SUM(ratingoutof10) as tot
FROM dbo.[@dog_rates]
GROUP BY doggo;
GO

## GROUP BY and AVG()

![group by and avg](images/group_avg.png)

In [ ]:
USE TSQLSchool;
GO
--Is the return value of this correct?
--Stormy has two ratingoutof10's: 13 and 14
SELECT
    doggo,
    AVG(ratingoutof10) as tot
FROM dbo.[@dog_rates]
GROUP BY doggo;
GO

In [ ]:
--What about this?
SELECT
    doggo,
    AVG(1.0 * ratingoutof10) as tot
FROM dbo.[@dog_rates]
GROUP BY doggo;
GO


--Return data type: numeric(38,6) - precision 38, scale 6 

# GROUP BY and HAVING

## Logical processing order 

* FROM
* ON
* JOIN
* WHERE
* <mark>GROUP BY</mark>
* WITH CUBE or WITH ROLLUP
* <mark>HAVING</mark>
* SELECT
* DISTINCT
* ORDER BY
* TOP

https://docs.microsoft.com/en-us/sql/t-sql/queries/select-transact-sql 

## GROUP BY and HAVING with MIN()

![group by and having](images/group_having_min.png)

In [ ]:
USE TSQLSchool;
GO
SELECT
    doggo,
    MIN(ratingoutof10) as tot
FROM dbo.[@dog_rates]
GROUP BY doggo
HAVING 
    MIN(ratingoutof10) < 14;
GO

## GROUP BY and HAVING with COUNT()

![group by and having](images/group_having_count.png)

In [ ]:
USE TSQLSchool;
GO
SELECT
    doggo
FROM dbo.[@dog_rates]
GROUP BY doggo
HAVING 
    COUNT(*) > 1;
GO

# CASE

## "CASE colname when" syntax

![case](images/case_colname_when.png)

In [ ]:
use TSQLSchool;
GO
SELECT
   doggo,
   CASE ratingoutof10
       WHEN 14 THEN 'boop'
       WHEN 15 THEN 'blep'
       ELSE 'h*ck'
   END AS correctedrating
FROM dbo.[@dog_rates];
GO

## "CASE when condition" syntax

![case](images/case_when_condition.png)

In [ ]:
USE TSQLSchool;
GO
SELECT
   doggo,
   CASE 
    WHEN ratingoutof10 < 14
    THEN 'h*ck'
   END AS correctedrating
FROM dbo.[@dog_rates];
GO

## "CASE when compound condition" syntax

![case](images/case_when_compoundcondition.png)

In [ ]:
USE TSQLSchool;
GO

SELECT
   doggo +
   CASE 
    WHEN doggo = ratedby
        and ratingoutof10 > 10
    THEN ' is proud puppers'
    ELSE '!'
   END AS sentence
FROM dbo.[@dog_rates];
GO


In [ ]:
--What if we forgot the "ELSE" ?

USE TSQLSchool;
GO

SELECT
   doggo +
   CASE 
    WHEN doggo = ratedby
        and ratingoutof10 > 10
    THEN ' is proud puppers'
    --ELSE '!'
   END AS sentence
FROM dbo.[@dog_rates];
GO

# GROUP BY and CASE

![case](images/group_case.png)

In [ ]:
USE TSQLSchool;
GO

SELECT
    doggo,
    SUM(
        CASE WHEN doggo = ratedby
            THEN 1
            ELSE 0
        END
    ) as selfratings
FROM dbo.[@dog_rates]
GROUP BY doggo;